<a href="https://colab.research.google.com/github/mickeykubo/fastai-v1/blob/master/layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Layers

In [1]:
!pip3 install https://download.pytorch.org/whl/cu80/torch-1.0.0-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision
!pip3 install fastai
!pip3 install Pillow==4.1.1
# fast.ai が準備した以下のコマンドだと1行でいける．
# 注意：Google Colabの環境が変わる可能性がある．
#curl https://course-v3.fast.ai/setup/colab | bash

This module contains many layer classes that we might be interested in using in our models. These layers complement the default [Pytorch layers](https://pytorch.org/docs/stable/nn.html) which we can also use as predefined layers.

In [0]:
from fastai.vision import *
from fastai.gen_doc.nbdoc import *

## Custom fastai modules

In [3]:
show_doc(AdaptiveConcatPool2d, title_level=3)

<h3 id="AdaptiveConcatPool2d"><code>class</code> <code>AdaptiveConcatPool2d</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L171" class="source_link">[source]</a></h3>

> <code>AdaptiveConcatPool2d</code>(<b>`sz`</b>:`Optional`\[`int`\]=<b><i>`None`</i></b>) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Layer that concats `AdaptiveAvgPool2d` and `AdaptiveMaxPool2d`.  

In [0]:
from fastai.gen_doc.nbdoc import *
from fastai.layers import * 

The output will be `2*sz`, or just 2 if `sz` is None.

The [`AdaptiveConcatPool2d`](/layers.html#AdaptiveConcatPool2d) object uses adaptive average pooling and adaptive max pooling and concatenates them both. We use this because it provides the model with the information of both methods and improves performance. This technique is called `adaptive` because it allows us to decide on what output dimensions we want, instead of choosing the input's dimensions to fit a desired output size.

Let's try training with Adaptive Average Pooling first, then with Adaptive Max Pooling and finally with the concatenation of them both to see how they fare in performance.

We will first define a [`simple_cnn`](/layers.html#simple_cnn) using [Adaptive Max Pooling](https://pytorch.org/docs/stable/nn.html#torch.nn.AdaptiveMaxPool2d) by changing the source code a bit.

In [0]:
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path)

In [0]:
def simple_cnn_max(actns:Collection[int], kernel_szs:Collection[int]=None,
               strides:Collection[int]=None) -> nn.Sequential:
    "CNN with `conv2d_relu` layers defined by `actns`, `kernel_szs` and `strides`"
    nl = len(actns)-1
    kernel_szs = ifnone(kernel_szs, [3]*nl)
    strides    = ifnone(strides   , [2]*nl)
    layers = [conv_layer(actns[i], actns[i+1], kernel_szs[i], stride=strides[i])
        for i in range(len(strides))]
    layers.append(nn.Sequential(nn.AdaptiveMaxPool2d(1), Flatten()))
    return nn.Sequential(*layers)

In [7]:
model = simple_cnn_max((3,16,16,2))
learner = Learner(data, model, metrics=[accuracy])
learner.fit(1)

epoch,train_loss,valid_loss,accuracy
1,0.106114,0.076576,0.980373


Now let's try with [Adaptive Average Pooling](https://pytorch.org/docs/stable/nn.html#torch.nn.AdaptiveAvgPool2d) now.

In [0]:
def simple_cnn_avg(actns:Collection[int], kernel_szs:Collection[int]=None,
               strides:Collection[int]=None) -> nn.Sequential:
    "CNN with `conv2d_relu` layers defined by `actns`, `kernel_szs` and `strides`"
    nl = len(actns)-1
    kernel_szs = ifnone(kernel_szs, [3]*nl)
    strides    = ifnone(strides   , [2]*nl)
    layers = [conv_layer(actns[i], actns[i+1], kernel_szs[i], stride=strides[i])
        for i in range(len(strides))]
    layers.append(nn.Sequential(nn.AdaptiveAvgPool2d(1), Flatten()))
    return nn.Sequential(*layers)

In [9]:
model = simple_cnn_avg((3,16,16,2))
learner = Learner(data, model, metrics=[accuracy])
learner.fit(1)

epoch,train_loss,valid_loss,accuracy
1,0.266963,0.213397,0.977920


Finally we will try with the concatenation of them both [`AdaptiveConcatPool2d`](/layers.html#AdaptiveConcatPool2d). We will see that, in fact, it increases our accuracy and decreases our loss considerably!

In [0]:
def simple_cnn(actns:Collection[int], kernel_szs:Collection[int]=None,
               strides:Collection[int]=None) -> nn.Sequential:
    "CNN with `conv2d_relu` layers defined by `actns`, `kernel_szs` and `strides`"
    nl = len(actns)-1
    kernel_szs = ifnone(kernel_szs, [3]*nl)
    strides    = ifnone(strides   , [2]*nl)
    layers = [conv_layer(actns[i], actns[i+1], kernel_szs[i], stride=strides[i])
        for i in range(len(strides))]
    layers.append(nn.Sequential(AdaptiveConcatPool2d(1), Flatten()))
    return nn.Sequential(*layers)

In [11]:
model = simple_cnn((3,16,16,2))
learner = Learner(data, model, metrics=[accuracy])
learner.fit(1)

epoch,train_loss,valid_loss,accuracy
1,0.139438,0.113247,0.978901


In [12]:
show_doc(Lambda, title_level=3)

<h3 id="Lambda"><code>class</code> <code>Lambda</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L10" class="source_link">[source]</a></h3>

> <code>Lambda</code>(<b>`func`</b>:`LambdaFunc`) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

An easy way to create a pytorch layer for a simple `func`.  

This is very useful to use functions as layers in our networks inside a [Sequential](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential) object. So, for example, say we want to apply a [log_softmax loss](https://pytorch.org/docs/stable/nn.html#torch.nn.functional.log_softmax) and we need to change the shape of our output batches to be able to use this loss. We can add a layer that applies the necessary change in shape by calling:

`Lambda(lambda x: x.view(x.size(0),-1))`

Let's see an example of how the shape of our output can change when we add this layer.

In [0]:
model = nn.Sequential(
    nn.Conv2d(3,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
)

model.cuda()

for xb, yb in data.train_dl:
    out = (model(*[xb]))
    print(out.size())
    break

torch.Size([64, 10, 1, 1])


In [0]:
model = nn.Sequential(
    nn.Conv2d(3,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    Lambda(lambda x: x.view(x.size(0),-1))
)

model.cuda()

for xb, yb in data.train_dl:
    out = (model(*[xb]))
    print(out.size())
    break

torch.Size([64, 10])


In [0]:
show_doc(Flatten)

<h2 id="Flatten"><code>class</code> <code>Flatten</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L23" class="source_link">[source]</a></h2>

> <code>Flatten</code>(**`full`**:`bool`=***`False`***) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Flatten `x` to a single dimension, often used at the end of a model. `full` for rank-1 tensor  

The function we build above is actually implemented in our library as [`Flatten`](/layers.html#Flatten). We can see that it returns the same size when we run it.

In [0]:
model = nn.Sequential(
    nn.Conv2d(3,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    Flatten(),
)

model.cuda()

for xb, yb in data.train_dl:
    out = (model(*[xb]))
    print(out.size())
    break

torch.Size([64, 10])


In [0]:
show_doc(PoolFlatten)

<h4 id="PoolFlatten"><code>PoolFlatten</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L34" class="source_link">[source]</a></h4>

> <code>PoolFlatten</code>() → [`Sequential`](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential)

Apply [`nn.AdaptiveAvgPool2d`](https://pytorch.org/docs/stable/nn.html#torch.nn.AdaptiveAvgPool2d) to `x` and then flatten the result.  

We can combine these two final layers ([AdaptiveAvgPool2d](https://pytorch.org/docs/stable/nn.html#torch.nn.AdaptiveAvgPool2d) and [`Flatten`](/layers.html#Flatten)) by using [`PoolFlatten`](/layers.html#PoolFlatten).

In [0]:
model = nn.Sequential(
    nn.Conv2d(3,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    PoolFlatten()
)

model.cuda()

for xb, yb in data.train_dl:
    out = (model(*[xb]))
    print(out.size())
    break

torch.Size([64, 10])


Another use we give to the Lambda function is to resize batches with [`ResizeBatch`](/layers.html#ResizeBatch) when we have a layer that expects a different input than what comes from the previous one.

In [0]:
show_doc(ResizeBatch)

<h4 id="ResizeBatch"><code>ResizeBatch</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L19" class="source_link">[source]</a></h4>

> <code>ResizeBatch</code>(**\*`size`**:`int`) → `Tensor`

Layer that resizes x to `size`, good for connecting mismatched layers.  

In [0]:
a = torch.tensor([[1., -1.], [1., -1.]])
print(a)

tensor([[ 1., -1.],
        [ 1., -1.]])


In [0]:
out = ResizeBatch(4)
print(out(a))

tensor([[ 1., -1.,  1., -1.]])


In [0]:
show_doc(Debugger, title_level=3)

<h3 id="Debugger"><code>class</code> <code>Debugger</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L180" class="source_link">[source]</a></h3>

> <code>Debugger</code>() :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

A module to debug inside a model.  

The debugger module allows us to peek inside a network while its training and see in detail what is going on. We can see inputs, outputs and sizes at any point in the network.

For instance, if you run the following:

``` python
model = nn.Sequential(
    nn.Conv2d(3,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    Debugger(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
)

model.cuda()

learner = Learner(data, model, metrics=[accuracy])
learner.fit(5)
```
... you'll see something like this:

```
/home/ubuntu/fastai/fastai/layers.py(74)forward()
     72     def forward(self,x:Tensor) -> Tensor:
     73         set_trace()
---> 74         return x
     75 
     76 class StdUpsample(nn.Module):

ipdb>
```

In [0]:
show_doc(PixelShuffle_ICNR, title_level=3)

<h3 id="PixelShuffle_ICNR"><code>class</code> <code>PixelShuffle_ICNR</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L196" class="source_link">[source]</a></h3>

> <code>PixelShuffle_ICNR</code>(**`ni`**:`int`, **`nf`**:`int`=***`None`***, **`scale`**:`int`=***`2`***, **`blur`**:`bool`=***`False`***, **`norm_type`**=***`<NormType.Weight: 3>`***, **`leaky`**:`float`=***`None`***) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Upsample by `scale` from `ni` filters to `nf` (default `ni`), using [`nn.PixelShuffle`](https://pytorch.org/docs/stable/nn.html#torch.nn.PixelShuffle), [`icnr`](/layers.html#icnr) init, and [`weight_norm`](https://pytorch.org/docs/stable/nn.html#torch.nn.weight_norm).  

In [0]:
show_doc(MergeLayer, title_level=3)

<h3 id="MergeLayer"><code>class</code> <code>MergeLayer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L132" class="source_link">[source]</a></h3>

> <code>MergeLayer</code>(**`dense`**:`bool`=***`False`***) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Merge a shortcut with the result of the module by adding them or concatenating thme if `dense=True`.  

In [0]:
show_doc(PartialLayer, title_level=3)

<h3 id="PartialLayer"><code>class</code> <code>PartialLayer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L161" class="source_link">[source]</a></h3>

> <code>PartialLayer</code>(**`func`**, **\*\*`kwargs`**) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Layer that applies `partial(func, **kwargs)`.  

In [0]:
show_doc(SigmoidRange, title_level=3)

<h3 id="SigmoidRange"><code>class</code> <code>SigmoidRange</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L153" class="source_link">[source]</a></h3>

> <code>SigmoidRange</code>(**`low`**, **`high`**) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Sigmoid module with range `(low,x_max)`  

In [0]:
show_doc(SequentialEx, title_level=3)

<h3 id="SequentialEx"><code>class</code> <code>SequentialEx</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L111" class="source_link">[source]</a></h3>

> <code>SequentialEx</code>(**\*`layers`**) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Like [`nn.Sequential`](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential), but with ModuleList semantics, and can access module input  

In [0]:
show_doc(SelfAttention, title_level=3)

<h3 id="SelfAttention"><code>class</code> <code>SelfAttention</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L63" class="source_link">[source]</a></h3>

> <code>SelfAttention</code>(**`n_channels`**:`int`) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Self attention layer for 2d.  

In [0]:
show_doc(BatchNorm1dFlat, title_level=3)

<h3 id="BatchNorm1dFlat"><code>class</code> <code>BatchNorm1dFlat</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L280" class="source_link">[source]</a></h3>

> <code>BatchNorm1dFlat</code>(**`num_features`**, **`eps`**=***`1e-05`***, **`momentum`**=***`0.1`***, **`affine`**=***`True`***, **`track_running_stats`**=***`True`***) :: [`BatchNorm1d`](https://pytorch.org/docs/stable/nn.html#torch.nn.BatchNorm1d)

[`nn.BatchNorm1d`](https://pytorch.org/docs/stable/nn.html#torch.nn.BatchNorm1d), but first flattens leading dimensions  

## Loss functions

In [0]:
show_doc(FlattenedLoss, title_level=3)

<h3 id="FlattenedLoss"><code>class</code> <code>FlattenedLoss</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L215" class="source_link">[source]</a></h3>

> <code>FlattenedLoss</code>(**`func`**, **\*`args`**, **`axis`**:`int`=***`-1`***, **`floatify`**:`bool`=***`False`***, **`is_2d`**:`bool`=***`True`***, **\*\*`kwargs`**)

Same as `func`, but flattens input and target.  

Create an instance of `func` with `args` and `kwargs`. When passing an output and target, it
- puts `axis` first in output and target with a transpose
- casts the target to `float` is `floatify=True`
- squeezes the `output` to two dimensions if `is_2d`, otherwise one dimension, squeezes the target to one dimension
- applied the instance of `func`.

In [0]:
show_doc(BCEFlat)

<h4 id="BCEFlat"><code>BCEFlat</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L241" class="source_link">[source]</a></h4>

> <code>BCEFlat</code>(**\*`args`**, **`axis`**:`int`=***`-1`***, **`floatify`**:`bool`=***`True`***, **\*\*`kwargs`**)

Same as [`nn.BCELoss`](https://pytorch.org/docs/stable/nn.html#torch.nn.BCELoss), but flattens input and target.  

In [0]:
show_doc(BCEWithLogitsFlat)

<h4 id="BCEWithLogitsFlat"><code>BCEWithLogitsFlat</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L237" class="source_link">[source]</a></h4>

> <code>BCEWithLogitsFlat</code>(**\*`args`**, **`axis`**:`int`=***`-1`***, **`floatify`**:`bool`=***`True`***, **\*\*`kwargs`**)

Same as [`nn.BCEWithLogitsLoss`](https://pytorch.org/docs/stable/nn.html#torch.nn.BCEWithLogitsLoss), but flattens input and target.  

In [0]:
show_doc(CrossEntropyFlat)

<h4 id="CrossEntropyFlat"><code>CrossEntropyFlat</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L233" class="source_link">[source]</a></h4>

> <code>CrossEntropyFlat</code>(**\*`args`**, **`axis`**:`int`=***`-1`***, **\*\*`kwargs`**)

Same as [`nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/nn.html#torch.nn.CrossEntropyLoss), but flattens input and target.  

In [0]:
show_doc(MSELossFlat)

<h4 id="MSELossFlat"><code>MSELossFlat</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L245" class="source_link">[source]</a></h4>

> <code>MSELossFlat</code>(**\*`args`**, **`axis`**:`int`=***`-1`***, **`floatify`**:`bool`=***`True`***, **\*\*`kwargs`**)

Same as [`nn.MSELoss`](https://pytorch.org/docs/stable/nn.html#torch.nn.MSELoss), but flattens input and target.  

In [0]:
show_doc(NoopLoss)

<h2 id="NoopLoss"><code>class</code> <code>NoopLoss</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L249" class="source_link">[source]</a></h2>

> <code>NoopLoss</code>() :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Just returns the mean of the `output`.  

In [0]:
show_doc(WassersteinLoss)

<h2 id="WassersteinLoss"><code>class</code> <code>WassersteinLoss</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L253" class="source_link">[source]</a></h2>

> <code>WassersteinLoss</code>() :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

For WGAN.  

## Helper functions to create modules

In [0]:
show_doc(bn_drop_lin, doc_string=False)

<h4 id="bn_drop_lin"><code>bn_drop_lin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L48" class="source_link">[source]</a></h4>

> <code>bn_drop_lin</code>(**`n_in`**:`int`, **`n_out`**:`int`, **`bn`**:`bool`=***`True`***, **`p`**:`float`=***`0.0`***, **`actn`**:`Optional`\[[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)\]=***`None`***)

The [`bn_drop_lin`](/layers.html#bn_drop_lin) function returns a sequence of [batch normalization](https://arxiv.org/abs/1502.03167), [dropout](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf) and a linear layer. This custom layer is usually used at the end of a model. 

`n_in` represents the number of size of the input `n_out` the size of the output, `bn` whether we want batch norm or not, `p` is how much dropout and `actn` is an optional parameter to add an activation function at the end.

In [0]:
show_doc(conv2d)

<h4 id="conv2d"><code>conv2d</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L81" class="source_link">[source]</a></h4>

> <code>conv2d</code>(**`ni`**:`int`, **`nf`**:`int`, **`ks`**:`int`=***`3`***, **`stride`**:`int`=***`1`***, **`padding`**:`int`=***`None`***, **`bias`**=***`False`***, **`init`**:`LayerFunc`=***`'kaiming_normal_'`***) → [`Conv2d`](https://pytorch.org/docs/stable/nn.html#torch.nn.Conv2d)

Create and initialize [`nn.Conv2d`](https://pytorch.org/docs/stable/nn.html#torch.nn.Conv2d) layer. `padding` defaults to `ks//2`.  

In [0]:
show_doc(conv2d_trans)

<h4 id="conv2d_trans"><code>conv2d_trans</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L86" class="source_link">[source]</a></h4>

> <code>conv2d_trans</code>(**`ni`**:`int`, **`nf`**:`int`, **`ks`**:`int`=***`2`***, **`stride`**:`int`=***`2`***, **`padding`**:`int`=***`0`***, **`bias`**=***`False`***) → [`ConvTranspose2d`](https://pytorch.org/docs/stable/nn.html#torch.nn.ConvTranspose2d)

Create [`nn.ConvTranspose2d`](https://pytorch.org/docs/stable/nn.html#torch.nn.ConvTranspose2d) layer.  

In [0]:
show_doc(conv_layer, doc_string=False)

<h4 id="conv_layer"><code>conv_layer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L94" class="source_link">[source]</a></h4>

> <code>conv_layer</code>(**`ni`**:`int`, **`nf`**:`int`, **`ks`**:`int`=***`3`***, **`stride`**:`int`=***`1`***, **`padding`**:`int`=***`None`***, **`bias`**:`bool`=***`None`***, **`is_1d`**:`bool`=***`False`***, **`norm_type`**:`Optional`\[[`NormType`](/layers.html#NormType)\]=***`<NormType.Batch: 1>`***, **`use_activ`**:`bool`=***`True`***, **`leaky`**:`float`=***`None`***, **`transpose`**:`bool`=***`False`***, **`init`**:`Callable`=***`'kaiming_normal_'`***, **`self_attention`**:`bool`=***`False`***)

The [`conv_layer`](/layers.html#conv_layer) function returns a sequence of [nn.Conv2D](https://pytorch.org/docs/stable/nn.html#torch.nn.Conv2d), [BatchNorm](https://arxiv.org/abs/1502.03167) and a ReLU or [leaky RELU](https://ai.stanford.edu/~amaas/papers/relu_hybrid_icml2013_final.pdf) activation function.

`n_in` represents the number of size of the input `n_out` the size of the output, `ks` kernel size, `stride` the stride with which we want to apply the convolutions. `bias` will decide if they have bias or not (if None, defaults to True unless using batchnorm). `norm_type` selects type of normalization (or `None`). If `leaky` is None, the activation is a standard `ReLU`, otherwise it's a `LeakyReLU` of slope `leaky`. Finally if `transpose=True`, the convolution is replaced by a `ConvTranspose2D`.

In [0]:
show_doc(embedding, doc_string=False)

<h4 id="embedding"><code>embedding</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L273" class="source_link">[source]</a></h4>

> <code>embedding</code>(**`ni`**:`int`, **`nf`**:`int`) → [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Create an [embedding layer](https://arxiv.org/abs/1711.09160) with input size `ni` and output size `nf`.

In [0]:
show_doc(relu)

<h4 id="relu"><code>relu</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L90" class="source_link">[source]</a></h4>

> <code>relu</code>(**`inplace`**:`bool`=***`False`***, **`leaky`**:`float`=***`None`***)

Return a relu activation, maybe `leaky` and `inplace`.  

In [0]:
show_doc(res_block)

<h4 id="res_block"><code>res_block</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L140" class="source_link">[source]</a></h4>

> <code>res_block</code>(**`nf`**, **`dense`**:`bool`=***`False`***, **`norm_type`**:`Optional`\[[`NormType`](/layers.html#NormType)\]=***`<NormType.Batch: 1>`***, **`bottle`**:`bool`=***`False`***, **\*\*`conv_kwargs`**)

Resnet block of `nf` features. `conv_kwargs` are passed to [`conv_layer`](/layers.html#conv_layer).  

In [0]:
show_doc(sigmoid_range)

<h4 id="sigmoid_range"><code>sigmoid_range</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L149" class="source_link">[source]</a></h4>

> <code>sigmoid_range</code>(**`x`**, **`low`**, **`high`**)

Sigmoid function with range `(low, high)`  

In [0]:
show_doc(simple_cnn)

<h4 id="simple_cnn"><code>simple_cnn</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L257" class="source_link">[source]</a></h4>

> <code>simple_cnn</code>(**`actns`**:`Collection`\[`int`\], **`kernel_szs`**:`Collection`\[`int`\]=***`None`***, **`strides`**:`Collection`\[`int`\]=***`None`***, **`bn`**=***`False`***) → [`Sequential`](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential)

CNN with [`conv_layer`](/layers.html#conv_layer) defined by `actns`, `kernel_szs` and `strides`, plus batchnorm if `bn`.  

## Initialization of modules

In [0]:
show_doc(batchnorm_2d)

<h4 id="batchnorm_2d"><code>batchnorm_2d</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L40" class="source_link">[source]</a></h4>

> <code>batchnorm_2d</code>(**`nf`**:`int`, **`norm_type`**:[`NormType`](/layers.html#NormType)=***`<NormType.Batch: 1>`***)

A batchnorm2d layer with `nf` features initialized depending on `norm_type`.  

In [0]:
show_doc(icnr)

<h4 id="icnr"><code>icnr</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L186" class="source_link">[source]</a></h4>

> <code>icnr</code>(**`x`**, **`scale`**=***`2`***, **`init`**=***`'kaiming_normal_'`***)

ICNR init of `x`, with `scale` and `init` function.  

In [0]:
show_doc(trunc_normal_)

<h4 id="trunc_normal_"><code>trunc_normal_</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L268" class="source_link">[source]</a></h4>

> <code>trunc_normal_</code>(**`x`**:`Tensor`, **`mean`**:`float`=***`0.0`***, **`std`**:`float`=***`1.0`***) → `Tensor`

Truncated normal initialization.  

In [0]:
show_doc(icnr)

<h4 id="icnr"><code>icnr</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L186" class="source_link">[source]</a></h4>

> <code>icnr</code>(**`x`**, **`scale`**=***`2`***, **`init`**=***`'kaiming_normal_'`***)

ICNR init of `x`, with `scale` and `init` function.  

In [0]:
show_doc(NormType)

<h2 id="NormType">`NormType`</h2>

> <code>Enum</code> = [Batch, BatchZero, Weight, Spectral]

An enumeration.  

## Undocumented Methods - Methods moved below this line will intentionally be hidden

In [0]:
show_doc(Debugger.forward)

<h4 id="Debugger.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L182" class="source_link">[source]</a></h4>

> <code>forward</code>(**`x`**:`Tensor`) → `Tensor`

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [0]:
show_doc(Lambda.forward)

<h4 id="Lambda.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L17" class="source_link">[source]</a></h4>

> <code>forward</code>(**`x`**)

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [0]:
show_doc(AdaptiveConcatPool2d.forward)

<h4 id="AdaptiveConcatPool2d.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L178" class="source_link">[source]</a></h4>

> <code>forward</code>(**`x`**)

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [0]:
show_doc(NoopLoss.forward)

<h4 id="NoopLoss.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L251" class="source_link">[source]</a></h4>

> <code>forward</code>(**`output`**, **\*`args`**)

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [0]:
show_doc(PixelShuffle_ICNR.forward)

<h4 id="PixelShuffle_ICNR.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L211" class="source_link">[source]</a></h4>

> <code>forward</code>(**`x`**)

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [0]:
show_doc(WassersteinLoss.forward)

<h4 id="WassersteinLoss.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L255" class="source_link">[source]</a></h4>

> <code>forward</code>(**`real`**, **`fake`**)

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [0]:
show_doc(MergeLayer.forward)

<h4 id="MergeLayer.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L138" class="source_link">[source]</a></h4>

> <code>forward</code>(**`x`**)

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [0]:
show_doc(SigmoidRange.forward)

<h4 id="SigmoidRange.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L159" class="source_link">[source]</a></h4>

> <code>forward</code>(**`x`**)

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [0]:
show_doc(MergeLayer.forward)

<h4 id="MergeLayer.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L138" class="source_link">[source]</a></h4>

> <code>forward</code>(**`x`**)

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [0]:
show_doc(SelfAttention.forward)

<h4 id="SelfAttention.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L72" class="source_link">[source]</a></h4>

> <code>forward</code>(**`x`**)

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [0]:
show_doc(SequentialEx.forward)

<h4 id="SequentialEx.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L117" class="source_link">[source]</a></h4>

> <code>forward</code>(**`x`**)

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [0]:
show_doc(SequentialEx.append)

<h4 id="SequentialEx.append"><code>append</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L128" class="source_link">[source]</a></h4>

> <code>append</code>(**`l`**)

In [0]:
show_doc(SequentialEx.extend)

<h4 id="SequentialEx.extend"><code>extend</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L129" class="source_link">[source]</a></h4>

> <code>extend</code>(**`l`**)

In [0]:
show_doc(SequentialEx.insert)

<h4 id="SequentialEx.insert"><code>insert</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L130" class="source_link">[source]</a></h4>

> <code>insert</code>(**`i`**, **`l`**)

In [0]:
show_doc(PartialLayer.forward)

<h4 id="PartialLayer.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L168" class="source_link">[source]</a></h4>

> <code>forward</code>(**`x`**)

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [0]:
show_doc(BatchNorm1dFlat.forward)

<h4 id="BatchNorm1dFlat.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L282" class="source_link">[source]</a></h4>

> <code>forward</code>(**`x`**)

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [0]:
show_doc(Flatten.forward)

<h4 id="Flatten.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L29" class="source_link">[source]</a></h4>

> <code>forward</code>(**`x`**)

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

## New Methods - Please document or move to the undocumented section